The MIT License

Copyright (c) 2020 Yeong-Dae Kwon
 
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
 
The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.
 
 
 
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.



In [ ]:
SAVE_FOLDER_NAME = "INFERENCE_00"
print(SAVE_FOLDER_NAME)


In [ ]:
from HYPER_PARAMS import *  # NOTE : You much edit HYPER_PARAMS to match the model you are loading
from TORCH_OBJECTS import *
import numpy as np
import time


In [ ]:

from source.travelling_saleman_problem import GROUP_ENVIRONMENT
import source.MODEL__Actor.grouped_actors as A_Module
from source.utilities import Average_Meter, augment_xy_data_by_8_fold, Get_Logger


In [ ]:
# Make Log File
logger, result_folder_path = Get_Logger(SAVE_FOLDER_NAME)

print(result_folder_path)



In [ ]:
# Load Model
actor_group = A_Module.ACTOR().to(device)

actor_model_save_path = './result/Saved_TSP100_Model/ACTOR_state_dic.pt'
actor_group.load_state_dict(torch.load(actor_model_save_path, map_location="cuda:0"))
actor_group.eval()

logger.info('==============================================================================')
logger.info('==============================================================================')
log_str = '  <<< MODEL: {:s} >>>'.format(actor_model_save_path)
logger.info(log_str)


In [ ]:
LOG_PERIOD_SEC = 10
DATASET_SIZE = 100 *1000


In [ ]:
TEST_BATCH_SIZE = 1024
eval_dist_AM_0 = Average_Meter()


logger.info('===================================================================')
log_str = 'Single Trajectory'
logger.info(log_str)

timer_start = time.time()
logger_start = time.time()


episode = 0
while True:

    seq = Tensor(np.random.rand(TEST_BATCH_SIZE, TSP_SIZE, 2))
    batch_s = seq.size(0)
    episode = episode + batch_s



    with torch.no_grad():
        
        env = GROUP_ENVIRONMENT(seq)
        group_s = 1
        group_state, reward, done = env.reset(group_size=group_s)
        actor_group.reset(group_state)

        # First Move is given
        first_action = LongTensor(np.arange(group_s))[None, :].expand(batch_s, group_s)
        group_state, reward, done = env.step(first_action)

        while not done:
            actor_group.update(group_state)
            action_probs = actor_group.get_action_probabilities()
            # shape = (batch, group, TSP_SIZE)
            action = action_probs.argmax(dim=2)
            # shape = (batch, group)
            group_state, reward, done = env.step(action)

    eval_dist_AM_0.push(-reward)  # reward was given as negative dist



    if (time.time()-logger_start > LOG_PERIOD_SEC) or (episode >= DATASET_SIZE):
        timestr = time.strftime("%H:%M:%S", time.gmtime(time.time()-timer_start))
        log_str = 'Ep:{:07d}({:5.1f}%)  T:{:s}  avg.dist:{:f}'\
            .format(episode, episode/DATASET_SIZE*100,
                    timestr, eval_dist_AM_0.peek())
        logger.info(log_str)
        logger_start = time.time()
        
    if episode >= DATASET_SIZE:
        break
    
    
logger.info('---------------------------------------------------')
logger.info('average = {}'.format(eval_dist_AM_0.result()))
logger.info('---------------------------------------------------')
logger.info('---------------------------------------------------')



In [ ]:
TEST_BATCH_SIZE = 1024
eval_dist_AM_0 = Average_Meter()

logger.info('===================================================================')
log_str = 'All Trajectory'
logger.info(log_str)


timer_start = time.time()
logger_start = time.time()
episode = 0
while True:

    seq = Tensor(np.random.rand(TEST_BATCH_SIZE, TSP_SIZE, 2))

    batch_s = seq.size(0)
    episode = episode + batch_s



    with torch.no_grad():
        
        env = GROUP_ENVIRONMENT(seq)
        group_s = TSP_SIZE
        group_state, reward, done = env.reset(group_size=group_s)
        actor_group.reset(group_state)

        # First Move is given
        first_action = LongTensor(np.arange(group_s))[None, :].expand(batch_s, group_s)
        group_state, reward, done = env.step(first_action)

        while not done:
            actor_group.update(group_state)
            action_probs = actor_group.get_action_probabilities()
            # shape = (batch, group, TSP_SIZE)
            action = action_probs.argmax(dim=2)
            # shape = (batch, group)
            group_state, reward, done = env.step(action)

    max_reward, _ = reward.max(dim=1)
    eval_dist_AM_0.push(-max_reward)  # reward was given as negative dist



    if (time.time()-logger_start > LOG_PERIOD_SEC) or (episode >= DATASET_SIZE):
        timestr = time.strftime("%H:%M:%S", time.gmtime(time.time()-timer_start))
        log_str = 'Ep:{:07d}({:5.1f}%)  T:{:s}  avg.dist:{:f}'\
            .format(episode, episode/DATASET_SIZE*100,
                    timestr, eval_dist_AM_0.peek())
        logger.info(log_str)
        logger_start = time.time()
        
    if episode >= DATASET_SIZE:
        break
    
logger.info('---------------------------------------------------')
logger.info('average = {}'.format(eval_dist_AM_0.result()))
logger.info('---------------------------------------------------')
logger.info('---------------------------------------------------')




In [ ]:
TEST_BATCH_SIZE = 256
eval_dist_AM_0 = Average_Meter()

logger.info('===================================================================')
log_str = 'Augmented Input 8 Fold'
logger.info(log_str)


timer_start = time.time()
logger_start = time.time()
episode = 0
while True:

    seq = Tensor(np.random.rand(TEST_BATCH_SIZE, TSP_SIZE, 2))

    batch_s = seq.size(0)
    episode = episode + batch_s
    
    
    seq_augmented = augment_xy_data_by_8_fold(seq)
    batch_s = 8*batch_s
    
    with torch.no_grad():

        env = GROUP_ENVIRONMENT(seq_augmented)
        group_s = TSP_SIZE
        group_state, reward, done = env.reset(group_size=group_s)
        actor_group.reset(group_state)

        # First Move is given
        first_action = LongTensor(np.arange(group_s))[None, :].expand(batch_s, group_s)
        group_state, reward, done = env.step(first_action)

        while not done:
            actor_group.update(group_state)
            action_probs = actor_group.get_action_probabilities()
            # shape = (batch, group, TSP_SIZE)
            action = action_probs.argmax(dim=2)
            # shape = (batch, group)
            group_state, reward, done = env.step(action)



    batch_s = round(batch_s/8)
    group_reward = reward.reshape(8, batch_s, TSP_SIZE)
    # shape = (8, batch, group)

    # max_reward, idx_dim_2 = group_reward.max(dim=2)
    # max_reward, idx_dim_0 = max_reward.max(dim=0)
    
    max_reward, _ = group_reward.max(dim=2)
    max_reward, _ = max_reward.max(dim=0)
    
    
    eval_dist_AM_0.push(-max_reward)  # reward was given as negative dist    

    if (time.time()-logger_start > LOG_PERIOD_SEC) or (episode >= DATASET_SIZE):
        timestr = time.strftime("%H:%M:%S", time.gmtime(time.time()-timer_start))
        log_str = 'Ep:{:07d}({:5.1f}%)  T:{:s}  avg.dist:{:f}'\
            .format(episode, episode/DATASET_SIZE*100,
                    timestr, eval_dist_AM_0.peek())
        logger.info(log_str)
        logger_start = time.time()
        
    if episode >= DATASET_SIZE:
        break
        
logger.info('---------------------------------------------------')
logger.info('average = {}'.format(eval_dist_AM_0.result()))
logger.info('---------------------------------------------------')
logger.info('---------------------------------------------------')


    